In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Calibration.Qselfcal import scatter_plot_FP, get_TEScoordinates_ONAFP
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
def get_dict(comm, key='in'):
    """QUBIC dictionary.

    Method to modify the qubic dictionary.

    Parameters
    ----------
    key : str, optional
        Can be "in" or "out".
        It is used to build respectively the instances to generate the TODs or to reconstruct the sky maps,
        by default "in".

    Returns
    -------
    dict_qubic: dict
        Modified QUBIC dictionary.

    """

    args = {
        "npointings": 1,
        "nf_recon": 2,
        "nf_sub": 10,
        "nside": nside,
        "MultiBand": True,
        "period": 1,
        "RA_center": 0,
        "DEC_center": -57,
        "filter_nu": 150 * 1e9,
        "noiseless": True,
        "comm": comm,
        "dtheta": 15,
        "nprocs_sampling": 1,
        "nprocs_instrument": comm.Get_size(),
        "photon_noise": False,
        "nhwp_angles": 1,
        "effective_duration150": 3,
        "effective_duration220": 3,
        "filter_relative_bandwidth": 0.25,
        "type_instrument": "two",
        "TemperatureAtmosphere150": None,
        "TemperatureAtmosphere220": None,
        "EmissivityAtmosphere150": None,
        "EmissivityAtmosphere220": None,
        "detector_nep": float(4.7e-17),
        "synthbeam_kmax": 1,
        "synthbeam_fraction": 1,
        "beam_shape": "gaussian",
        "debug":False,
        "kind": "IQU",
    }

    ### Get the default dictionary
    dictfilename = "dicts/pipeline_demo.dict"
    dict_qubic = qubicDict()
    dict_qubic.read_from_file(dictfilename)

    for i in args.keys():

        dict_qubic[str(i)] = args[i]

    return dict_qubic

In [ ]:
# Build MPI communicator and qubic dict
comm = MPI.COMM_WORLD
dict_qubic = get_dict(comm)

In [ ]:
dict_qubic

In [ ]:
dict_qubic['center_detector'] = False

In [ ]:
dict_qubic["use_synthbeam_fits_file"] = False
print(dict_qubic["use_synthbeam_fits_file"])
print(dict_qubic["synthbeam"])
print(dict_qubic["primbeam"])

In [ ]:
# Build scanning strategy
sampling = get_pointing(dict_qubic)

# Build QubicInstrument
q = QubicInstrument(dict_qubic)

In [ ]:
# Number of sub-acquisitions
nsub = 20

In [ ]:
# Initialize the Qubic multi-frequency acquisition
# I multiply by 2 Nsub to have Nsub bands in each band, the rest of the code will only use the first Nsub bands (meaning the ones in the 150 GHz band)
acq = QubicMultiAcquisitions(dict_qubic, nsub*2, 2, sampling=sampling)

# Build a list that contains all sub-ProjectionOperators, meaning the ProjectionOperators for each sub-bands
Proj_list = []
for i in range(2*nsub):
    #Proj_list.append(acq.subacqs[i].get_projection_operator())*
    print(acq.H[i].operands[-1])
    Proj_list.append(acq.H[i].operands[-1])

In [ ]:
print(Proj_list[0].shapein, Proj_list[0].shapeout)
print(Proj_list[0].matrix.data.r11.shape)

In [ ]:
thetas, phis = np.zeros((len(acq.allnus), (2*dict_qubic['synthbeam_kmax'] + 1)**2)), np.zeros((len(acq.allnus), (2*dict_qubic['synthbeam_kmax'] + 1)**2))

Qacq = acq.subacqs 
idet = 0
kmax = dict_qubic['synthbeam_kmax']

for i in range(len(acq.allnus)):
    theta, phi = Qacq[i].instrument._peak_angles_kmax(kmax, Qacq[i].instrument.horn.spacing, Qacq[i].instrument.horn.angle, acq.allnus[i] * 1e9, Qacq[i].instrument.detector.center)
    thetas[i] = theta[idet]
    phis[i] = phi[idet]

In [ ]:
plt.plot(acq.allnus, np.degrees(thetas[:, 0]), 'o', label='theta')
plt.plot(acq.allnus, np.degrees(phis[:, 0]), 'o', label='phi')
plt.legend()
plt.xlabel('Frequency [GHz]')
plt.ylabel('Angle [deg]')

In [ ]:
hp.gnomview(np.zeros(hp.nside2npix(nside)), rot=[0,90], reso=20, title="Synth Beam")
hp.projplot(thetas[0], phis[0], 'rx', lonlat=False)

In [ ]:
npix = hp.nside2npix(nside)
hp.gnomview(np.zeros(npix), rot=[0,90], reso=15, title=f"Synth Beam - Det {idet}")

for i in range(len(acq.allnus[:nsub])):
    hp.projplot(thetas[i], phis[i], 'rx', lonlat=False)#, label='%.2f GHz' % acq.allnus[i])

In [ ]:
prim1_beam = np.zeros(npix)

# Compute Prim1ary Beam
prim1 = acq.multiinstrument[0].primary_beam
for i in range(npix):
    theta, phi = hp.pix2ang(nside, i)
    prim1_beam[i] = prim1(theta, phi)

hp.gnomview(prim1_beam, rot=[0,90], reso=20, title=f"Primary Beam {acq.allnus[0].round(2)} GHz - Synth Beam Det {idet}")
for i in range(len(acq.allnus[:nsub])):
    hp.projplot(thetas[i], phis[i], 'rx', lonlat=False)#, label='%.2f GHz' % acq.allnus[i])

# Build TOD

In [ ]:
# Create uniform maps of "1"
input_maps = np.ones((2*nsub, hp.nside2npix(nside), 3))

# Build the list of sub-TODs
tod = []
for i in range(2*nsub):
    tod.append(Proj_list[i](input_maps[i]))

In [ ]:
print("Number of sub-TODs: ", len(tod))
print("sub-TODs shape: ", tod[0].shape, " = (Ndet, Nsample, Nstk)")

# Comparing TODs for different Nsub

In [ ]:
nsub_max = 50
nsub_list = np.arange(10, nsub_max+1, 10)

TOD_list = []
freq_list = []

for insub in nsub_list:
    input_maps = np.ones((insub, hp.nside2npix(nside), 3))
    
    Qacq = QubicMultiAcquisitions(dict_qubic, 2*insub, 2, sampling=sampling)
    H = Qacq.H[:int(insub/2)]
    
    tod_proj = []
    for i in range(int(insub/2)):
        P = H[i].operands[-1]
        print(P)
        tod_proj.append(P(input_maps[i]))
    TOD_list.append(tod_proj)
    freq_list.append(Qacq.allnus)

In [ ]:
# TOD_list[len(nsub_list)][Nsub][Nsub, Npointings, Nstk]
print(TOD_list[0][0].shape)

idet, isamples = 0, 0

for i in range(len(nsub_list)):
    plt.plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, isamples, 0], '.r', label="I" if i==0 else "")
    plt.plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, isamples, 1], 'xb', label="Q" if i==0 else "")
    plt.plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, isamples, 2], '.g', label="U" if i==0 else "")
plt.legend()
plt.xlabel("Nsub")
plt.ylabel("TOD")

In [ ]:
### Sum detectors

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
for i in range(len(nsub_list)):
    ax[0].plot(nsub_list[i], np.sum(np.mean(TOD_list[i], axis=0), axis=0)[0, 0], '.r')
    ax[1].plot(nsub_list[i], np.sum(np.mean(TOD_list[i], axis=0), axis=0)[0, 1], 'xb')
    ax[2].plot(nsub_list[i], np.sum(np.mean(TOD_list[i], axis=0), axis=0)[0, 2], '.g')

ax[0].set_title('Intensity')
ax[1].set_title('Polarization Q')
ax[2].set_title('Polarization U')
for iax in range(ax.size):
    ax[iax].set_xlabel('Nsub')
    #ax[iax].set_yscale('log')
ax[0].set_ylabel('TOD')
fig.suptitle('TOD sum over detectors - One Time Sample')

In [ ]:
### One detector

idet = 0

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
for i in range(len(nsub_list)):
    ax[0].plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, 0, 0], '.r')
    ax[1].plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, 0, 1], 'xb')
    ax[2].plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet, 0, 2], '.g')

ax[0].set_title('Intensity')
ax[1].set_title('Polarization Q')
ax[2].set_title('Polarization U')
for iax in range(ax.size):
    ax[iax].set_xlabel('Nsub')
    #ax[iax].set_yscale('log')
ax[0].set_ylabel('TOD')
fig.suptitle('TOD sum over detectors - One Time Sample')

In [ ]:
### On the focal plane

# Compute the position of the TES in the focal plane
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

fig, axs = plt.subplots(3, len(nsub_list), figsize=(30,20))
for i in range(len(nsub_list)):
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 0], frame='ONAFP', s=150, title=f'I - Nsub = {nsub_list[i]}', fig=fig, ax = axs[0][i], unit=None, cbar = True)
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 1], frame='ONAFP', s=150, title=f'Q - Nsub = {nsub_list[i]}', fig=fig, ax = axs[1, i], unit=None, cbar = True)
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 2], frame='ONAFP', s=150, title=f'U - Nsub = {nsub_list[i]}', fig=fig, ax = axs[2, i], unit=None, cbar = True)

fig.suptitle("TOD at one time sample on the focal plane", fontsize=20)

In [ ]:
### On the focal plane

fig, axs = plt.subplots(3, len(nsub_list), figsize=(30,20))
for i in range(1, len(nsub_list)):
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 0] - np.mean(TOD_list[i], axis=0)[:, 0, 0], frame='ONAFP', s=150, title=f'I - Nsub = {nsub_list[i]}', fig=fig, ax = axs[0][i], unit=None, cbar = True)
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 0] - np.mean(TOD_list[i], axis=0)[:, 0, 1], frame='ONAFP', s=150, title=f'Q - Nsub = {nsub_list[i]}', fig=fig, ax = axs[1, i], unit=None, cbar = True)
    scatter_plot_FP(q, xTES, yTES, np.mean(TOD_list[i], axis=0)[:, 0, 0] - np.mean(TOD_list[i], axis=0)[:, 0, 2], frame='ONAFP', s=150, title=f'U - Nsub = {nsub_list[i]}', fig=fig, ax = axs[2, i], unit=None, cbar = True)

fig.suptitle("Difference with TOD Nsub = 10 on the focal plane", fontsize=20)

In [ ]:
### All detectors
for i in range(len(nsub_list)):
    for j in range(992):
        plt.plot(nsub_list[i], np.sum(TOD_list[i], axis=0)[j, 0, 0], '.r')


plt.xlabel('Nsub')
plt.ylabel('TOD')
plt.title('TOD  for all detectors - One Time Sample - Intensity')

In [ ]:
for i in range(len(nsub_list)):
    for j in range(len(TOD_list[i])):
        plt.plot(nsub_list[i], TOD_list[i][j][0, 0, 0], '.r', label="I" if (i==0 and j==0) else "")
        plt.plot(nsub_list[i], TOD_list[i][j][0, 0, 1], 'xb', label="Q" if (i==0 and j==0) else "")
        plt.plot(nsub_list[i], TOD_list[i][j][0, 0, 2], '.g', label="U" if (i==0 and j==0) else "")
plt.legend()
plt.xlabel("Nsub")
plt.ylabel("TOD")

# Plot sub-TODs

In [ ]:
# Plot of the TOD, taking the mean over the detectors

for i in range(nsub):
    plt.plot(acq.allnus[i], np.mean(tod[i][:, 0, 0]), '.r', label="I" if i==0 else "")
    plt.plot(acq.allnus[i], np.mean(tod[i][:, 0, 1]), 'xb', label="Q" if i==0 else "")
    plt.plot(acq.allnus[i], np.mean(tod[i][:, 0, 2]), '.g', label="U" if i==0 else "")
plt.legend()
plt.xlabel("Frequency [GHz]")
plt.ylabel(r"$P_{roj}(map)$")
plt.title("TODs")
plt.show()

plt.figure()
plt.title("Zoom on Polarisation")
for i in range(nsub):
    plt.plot(acq.allnus[i], np.mean(tod[i][:, 0, 1]), 'xb', label="Q" if i==0 else "")
    plt.plot(acq.allnus[i], np.mean(tod[i][:, 0, 2]), '.g', label="U" if i==0 else "")
plt.legend()
plt.xlabel("Frequency [GHz]")
plt.ylabel(r"$P_{roj}(map)$")

In [ ]:
for i in range(nsub):
    plt.plot(Proj_list[i].matrix.data.r11[0], '.r', label="A" if i==0 else "")
    plt.plot(Proj_list[i].matrix.data.r22[0], 'xb', label=r"$A\cos(2\alpha)$" if i==0 else "")
    plt.plot(Proj_list[i].matrix.data.r32[0], '.g', label=r"$-A\sin(2\alpha)$" if i==0 else "")
plt.legend()
plt.xlabel('Peak index')
plt.ylabel('Projection matrix element')

plt.figure()
colors = plt.cm.viridis(np.linspace(0, 1, nsub))
for i in range(nsub):
    for ipeak in range((2*dict_qubic['synthbeam_kmax']+1)**2):
        plt.scatter(ipeak, Proj_list[i].matrix.data.r11[0, ipeak], c=colors[i], marker = '.', label="A" if (i==0 and ipeak == 0) else "")
        plt.scatter(ipeak, Proj_list[i].matrix.data.r22[0, ipeak], c=colors[i], marker = 'x', label=r"$A\cos(2\alpha)$" if (i==0 and ipeak == 0) else "")
        plt.scatter(ipeak, Proj_list[i].matrix.data.r32[0, ipeak], c=colors[i], marker = 'o', label=r"$-A\sin(2\alpha)$" if (i==0 and ipeak == 0) else "")
plt.legend()
plt.colorbar()  # Add colorbar
plt.xlabel('Peak index')
plt.ylabel('Projection matrix element')

In [ ]:
import matplotlib as mpl

fig, ax = plt.subplots()

cmap = plt.cm.jet
norm = mpl.colors.Normalize(vmin=np.min(acq.allnus[:nsub]), vmax=np.max(acq.allnus[:nsub]))
colors = cmap(norm(acq.allnus[:nsub]))

for i in range(nsub):
    for ipeak in range((2 * dict_qubic['synthbeam_kmax'] + 1) ** 2):
        ax.scatter(ipeak, Proj_list[i].matrix.data.r11[0, ipeak],
                   color=colors[i],
                   marker='.',
                   label="A" if (i == 0 and ipeak == 0) else "")
        ax.scatter(ipeak, Proj_list[i].matrix.data.r22[0, ipeak],
                   color=colors[i],
                   marker='x',
                   label=r"$A\cos(2\alpha)$" if (i == 0 and ipeak == 0) else "")
        ax.scatter(ipeak, Proj_list[i].matrix.data.r32[0, ipeak],
                   color=colors[i],
                   marker='^',
                   label=r"$-A\sin(2\alpha)$" if (i == 0 and ipeak == 0) else "")

ax.legend()
ax.set_xlabel('Peak index')
ax.set_ylabel('Projection matrix element')

sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([]) 

cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Frequency (GHz)')

In [ ]:
base_colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]

color_variations = {0: [], 1: [], 2: []}
for i in range(nsub):
    factor = 0.5 + 0.5 * i / (nsub - 1)  # varies from 0.5 to 1
    for idx, base in enumerate(base_colors):
        varied_color = tuple(np.clip(np.array(base) * factor, 0, 1))
        color_variations[idx].append(varied_color)

plt.figure()

for i in range(nsub):
    for ipeak in range((2 * dict_qubic['synthbeam_kmax'] + 1) ** 2):
        plt.scatter(ipeak, Proj_list[i].matrix.data.r11[0, ipeak],
                    color=color_variations[0][i],
                    label="A" if (i == 0 and ipeak == 0) else "")
        plt.scatter(ipeak, Proj_list[i].matrix.data.r22[0, ipeak],
                    color=color_variations[1][i],
                    label=r"$A\cos(2\alpha)$" if (i == 0 and ipeak == 0) else "")
        plt.scatter(ipeak, Proj_list[i].matrix.data.r32[0, ipeak],
                    color=color_variations[2][i],
                    label=r"$-A\sin(2\alpha)$" if (i == 0 and ipeak == 0) else "")

plt.legend()
plt.xlabel('Peak Index')
plt.ylabel('Projection matrix element')
plt.title('Projection matrix elements for different sub-bands. The darker the color, the higher the frequency.')
plt.show()

# Relative difference between succesive sub-TODs

## Mean on detectors

In [ ]:
plt.figure()
for i in range(nsub-1):
    diff_I = np.abs((tod[i+1][:, 0, 0] - tod[i][:, 0, 0])/tod[i][:, 0, 0])
    mean_diff_I = np.mean(diff_I)
    plt.plot(acq.allnus[i], mean_diff_I, '.r', label="I" if i==0 else "")
    diff_Q = np.abs((tod[i+1][:, 0, 1] - tod[i][:, 0, 1])/tod[i][:, 0, 1])
    mean_diff_Q = np.mean(diff_Q)
    plt.plot(acq.allnus[i], mean_diff_Q, 'xb', label="Q" if i==0 else "")
    diff_U = np.abs((tod[i+1][:, 0, 2] - tod[i][:, 0, 2])/tod[i][:, 0, 2])
    mean_diff_U = np.mean(diff_U)
    plt.plot(acq.allnus[i], mean_diff_U, '.g', label="U" if i==0 else "")
    
plt.title(r"$H = \mathcal{P}_{roj}$")
plt.ylabel(r'$\frac{| H(map)^{i+1} - H(map)^i |}{| H(map)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')
plt.legend()

In [ ]:
# Intensity
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 0] - tod[i][:, 0, 0])/tod[i][:, 0, 0])
    mean_diff = np.mean(diff)
    plt.plot(acq.allnus[i], mean_diff, '.')
    
plt.title(r"$H = \mathcal{P}_{roj}$ - Intensity map")
plt.ylabel(r'$\frac{| H(Imap)^{i+1} - H(Imap)^i |}{| H(Imap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
# Polarisation
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 1] - tod[i][:, 0, 1])/tod[i][:, 0, 1])
    mean_diff = np.mean(diff)
    plt.plot(acq.allnus[i], mean_diff, '.')
    
plt.title(r"$H = \mathcal{P}_{roj}$ - Q map")
plt.ylabel(r'$\frac{| H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 2] - tod[i][:, 0, 2])/tod[i][:, 0, 2])
    mean_diff = np.mean(diff)
    plt.plot(acq.allnus[i], mean_diff, '.')
    
plt.title(r"$H = \mathcal{P}_{roj}$ - U map")
plt.ylabel(r'$\frac{| H(Umap)^{i+1} - H(Umap)^i |}{| H(Umap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

## For each detectors

In [ ]:
plt.figure()

# Range of detectors for which we want to plot the difference
idet_min, idet_max = 200, 210

for idet in range(idet_min, idet_max):
    for i in range(nsub-1):
        diff = (tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1] #np.abs((tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1])
        plt.plot(acq.allnus[i], diff, '.b')
        
    plt.title(r"$H = \mathcal{P}_{roj}$ - Q map - " + f'Detector index = {idet}')
    plt.ylabel(r'$\frac{ H(Qmap)^{i+1} - H(Qmap)^i }{ H(Qmap)^{i}}$', fontsize=15, labelpad=10)
    plt.xlabel('Frequency (GHz)')
    plt.show()

# Relative difference on the focal plane

Plot the difference between two successive tods, at a given time, on the focal plane. The polarisation case seems very strange with respect to the intensity case.

In [ ]:
# Compute the position of the TES in the focal plane
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

In [ ]:
# Sub-bands indices for the comparison
ind_a, ind_b = 3, 2

# Intensity
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ind_a]-tod[ind_b])/tod[ind_b]*100
print(relative_diff.shape)
scatter_plot_FP(q, xTES, yTES, np.mean(relative_diff[...,0], axis=1), frame='ONAFP', s=150, title=r'Intensity map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
# Polarisation Q
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (np.abs(tod[ind_a]-tod[ind_b]))/tod[ind_b]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $\frac{ \mathcal{P}_{roj}(Qmap)^{n+1} - \mathcal{P}_{roj}(Qmap)^{n} }{ \mathcal{P}_{roj}(Qmap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
# Polarisation U
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ind_a]-tod[ind_b])/tod[ind_b]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 2], frame='ONAFP', s=150, title=r'U map - $\frac{ \mathcal{P}_{roj}(Umap)^{n+1} - \mathcal{P}_{roj}(Umap)^{n} }{ \mathcal{P}_{roj}(Umap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

# Peak Amplitude evolution with frequency

P.matrix.data is build such that you have all the points for the same detector at all the different time samples, then the next detector at all the different time samples, etc.

In [ ]:
# Indices to compute the amplitude of peaks
peak_index = 0
N_sample = 0
time_index = 0
det_index = 203

# Plot the amplitude of the peak for each sub-band
for peak_index in range((2*dict_qubic['synthbeam_kmax']+1)**2 - 1):
    plt.figure()
    peak_amplitude = []
    for i in range(nsub):
        peak_amplitude.append(Proj_list[i].matrix.data.r11[(det_index*N_sample)+time_index, peak_index])

    plt.plot(acq.allnus[:nsub], peak_amplitude)
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Peak Amplitude")
    plt.title(f"Detector Index = {det_index} | Time Sample Index = {time_index} | Peak Index = {peak_index}")
    plt.show()

# Comparing Primary and Synthesized Beams evolution with frequency

In [ ]:
# Indices for the plot
idet = 0
N_sample = 0
time_index = 0

# Store the primary beam for each sub-band
primbeam_frequencies = [acq.multiinstrument[i].primary_beam for i in range(nsub)]

for ipeak in range((2*dict_qubic['synthbeam_kmax']+1)**2):
    peak_amplitude = [] 
    fig, ax1 = plt.subplots()
    for i in range(nsub):
        # Sub-acquisition for the i-th sub-band
        subacq_i = acq.subacqs[i]

        # Compute the peaks position and ampplitude
        th, ph, v = subacq_i.instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        subacq_i.instrument.detector.center,
                                        subacq_i.instrument.synthbeam,
                                        subacq_i.instrument.horn,
                                        subacq_i.instrument.primary_beam)

        # Plot the amplitude of the primary beam at peaks' position
        ax1.plot(acq.allnus[i], primbeam_frequencies[i](th[idet, ipeak], ph[idet, ipeak]), '.b')
        
        # Store the amplitude of the peak
        # Note: I checked that both gives the same result 
        peak_amplitude.append(v[idet, ipeak])
        #peak_amplitude.append(Proj_list[i].matrix.data.r11[(idet*N_sample)+time_index, ipeak])

    ax1.set_xlabel('Frequency (GHz)')
    ax1.set_ylabel('Primary beam', color = 'blue')
    ax1.set_title('Primary beam and peak amplitude at peak %d for detector %d' % (ipeak, idet))

    ax2 = ax1.twinx()
    ax2.plot(acq.allnus[:nsub], peak_amplitude, '.r')
    ax2.set_ylabel("Peak Amplitude", color = 'r')
    plt.show()